In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
client_id = 'CLIENT_ID'
client_secret = 'CLIENT_SECRET'
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
from pydub import AudioSegment
from pydub.silence import detect_leading_silence

trim_leading_silence = lambda x: x[detect_leading_silence(x) :]
trim_trailing_silence = lambda x: trim_leading_silence(x.reverse()).reverse()
strip_silence = lambda x: trim_trailing_silence(trim_leading_silence(x))

In [ ]:
import os

genres = os.listdir("./MIDIs")
instruments = os.listdir("./instruments")

In [ ]:
key_dict = {
    0: 'C', 1: 'C#',
    2: 'D', 3: 'D#',
    4: 'E',
    5: 'F', 6: 'F#',
    7: 'G', 8: 'G#',
    9: 'A', 10: 'A#',
    11: 'B'
}

mode_dict = {
    0: 'Minor', 1: 'Major'
}

In [ ]:
import random
from midi2audio import FluidSynth
from pedalboard import Pedalboard, Compressor, Phaser, Chorus, Bitcrush, Delay, Reverb
from pedalboard.io import AudioFile
import json

In [ ]:
os.makedirs('tmp', exist_ok=True)
os.makedirs('renders', exist_ok=True)
id = 0
for _ in range(1):
    # Pick song
    general_genre = random.choice(genres)
    specific_genre = random.choice(os.listdir(f"./MIDIs/{general_genre}"))
    artist = random.choice(os.listdir(f"./MIDIs/{general_genre}/{specific_genre}"))
    song = random.choice(os.listdir(f"./MIDIs/{general_genre}/{specific_genre}/{artist}"))

    # Render MIDI
    instrument = random.choice(instruments)
    fs = FluidSynth(f'./instruments/{instrument}', sample_rate=44100)
    fs.midi_to_audio(f'./MIDIs/{general_genre}/{specific_genre}/{artist}/{song}',
                     './tmp/render.wav')

    # Find it on Spotify
    spotify_track = sp.search(artist + " " + song[:-4], limit=1, type='track')
    if len(spotify_track['tracks']['items']) == 0: continue
    track_id = spotify_track['tracks']['items'][0]['id']

    # Get Spotify's Audio Features
    spotify_features = sp.audio_features([track_id])
    # acousticness = spotify_features[0]['acousticness'] # (maybe) Doesn't make sense, depends heavily on all instruments
    danceability = spotify_features[0]['danceability'] # Might depend on a lot of other instruments
    energy = spotify_features[0]['energy'] # Might depend on a lot of other instruments, mostly drums, but ok
    # instrumentalness = spotify_features[0]['instrumentalness'] # Doesn't make sense, it'll be just piano

    # Get Spotify's Audio Analysis
    spotify_analysis = sp.audio_analysis(track_id)
    audio = AudioSegment.from_wav('./tmp/render.wav')
    for section in spotify_analysis["sections"]:
        # Cut section of rendered audio
        start = section['start']*1000
        if (start > len(audio)): break # If the piano part ends earlier

        end = section['start']*1000 + section['duration']*1000
        if (end > len(audio)): end = len(audio)
        cut_audio = audio[start:end]
        
        # Remove start and end silence
        cut_audio = strip_silence(cut_audio)
        if len(cut_audio) < 4000: continue # If it's shorter than 4s don't save it

        cut_audio.export('./tmp/render_cut.wav', format='wav')

        data = {
            'song': song[:-4],
            'artist': artist,
            'instrument': instrument[:-4],
            'genre': specific_genre,
            'tempo': int(section['tempo']),
            'key': key_dict[section['key']],
            'mode': mode_dict[section['mode']],
            'energy': energy,
            'danceability': danceability,
            'pedals': []
        }

        # Add filters
        board = Pedalboard([Compressor()])
        num_pedals = random.choices([0, 1, 2, 3], weights=[40, 40, 15, 5], k=1)[0]
        pedals = random.sample([0, 1, 2, 3, 4], num_pedals)
        for pedal in pedals:
            match pedal:
                case 0:
                    rate_hz = random.uniform(0.1, 10.0)
                    depth = random.uniform(0.0, 1.0)
                    centre_frequency_hz = random.uniform(100.0, 10000.0)
                    feedback = 0.0
                    mix = 0.5
                    board.append(Phaser(rate_hz=rate_hz, depth=depth, centre_frequency_hz=centre_frequency_hz, feedback=feedback, mix=mix))
                    data['pedals'].append({
                        "name": "Phaser",
                        "rate_hz": rate_hz,
                        "depth": depth,
                        "centre_frequency_hz": centre_frequency_hz,
                    })
                case 1:
                    rate_hz = random.uniform(0.1, 10.0)
                    depth = random.uniform(0.0, 1.0)
                    centre_delay_ms = random.uniform(1.0, 50.0)
                    feedback = 0.0
                    mix = 0.5
                    board.append(Chorus(rate_hz=rate_hz, depth=depth, centre_delay_ms=centre_delay_ms, feedback=feedback, mix=mix))
                    data['pedals'].append({
                        "name": "Chorus",
                        "rate_hz": rate_hz,
                        "depth": depth,
                        "centre_delay_ms": centre_delay_ms,
                    })
                case 2:
                    bit_depth = 8 + 8*random.randint(0, 1) # 8 or 16
                    board.append(Bitcrush(bit_depth=bit_depth))
                    data['pedals'].append({
                        'name': 'Bitcrush',
                        'bit_depth': bit_depth
                    })
                case 3:
                    delay_seconds = random.uniform(0.01, 5.0)
                    feedback = 0.0
                    mix = 0.5
                    board.append(Delay(delay_seconds=delay_seconds, feedback=feedback, mix=mix))
                    data['pedals'].append({
                        "name": "Delay",
                        "delay_seconds": delay_seconds
                    })
                case 4:
                    room_size = random.uniform(0.0, 1.0)
                    damping = random.uniform(0.0, 1.0)
                    wet_level = random.uniform(0.0, 1.0)
                    dry_level = random.uniform(0.0, 1.0)
                    width = random.uniform(0.0, 1.0)
                    freeze_mode = random.uniform(0.0, 1.0)
                    board.append(Reverb(room_size=room_size, damping=damping, wet_level=wet_level, dry_level=dry_level, width=width, freeze_mode=freeze_mode))
                    data['pedals'].append({
                        "name": "Reverb",
                        "room_size": room_size,
                        "damping": damping,
                        "wet_level": wet_level,
                        "dry_level": dry_level,
                        "width": width,
                        "freeze_mode": freeze_mode
                    })


        with AudioFile('./tmp/render_cut.wav') as f:
            with AudioFile(f'./renders/{id:07}.wav', 'w', f.samplerate, f.num_channels) as o:
                while f.tell() < f.frames:
                    chunk = f.read(f.samplerate)
                    effected = board(chunk, f.samplerate, reset=False)
                    o.write(effected)

        # Save metadata
        with open(f'./renders/{id:07}.json', 'w') as json_file:
            json.dump(data, json_file)
        
        id += 1

os.remove(f'./tmp/render_cut.wav')
os.remove(f'./tmp/render.wav')
os.rmdir('tmp')

In [ ]:
import soundfile as sf
import pyloudnorm as pyln

In [ ]:
# Normalize renders loudness
for file in os.listdir('renders'):
    if file.endswith('.wav'):
        data, rate = sf.read(f'renders/{file}')
        meter = pyln.Meter(rate) # create BS.1770 meter
        loudness = meter.integrated_loudness(data)
        loudness_normalized_audio = pyln.normalize.loudness(data, loudness, -12.0)
        sf.write(f'renders/{file}', loudness_normalized_audio, rate)